# __Segmenting_and_Clustering_Neighborhoods_in_Toronto_Assignment_Notebook3__

## Part 1: Extract data from wikipedia, data wrangling and cleaning

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Scrape the wikipedia page and extract the Toronto table
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

table = soup.find('table', attrs= {'class': "wikitable sortable"})
table_row = table.find_all('tr')

res = []
for tr in table_row:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighborhood"])

# Find the index of row which Borough = Not assigned and drop those rows
ind = df[df['Borough'] == 'Not assigned'].index
df.drop(ind, axis = 0, inplace = True)

# Combine rows containing same PostalCode with the neighborhoods separated with a comma
df_clean = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

# Clean the column 'Neighborhood' for the case that a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough
x = np.linspace(0,len(df_clean['Neighborhood'])-1, len(df_clean['Neighborhood'])).astype(int)
for n in x:
    if df_clean['Neighborhood'][n] == 'Not assigned':
        df_clean['Neighborhood'][n] = df_clean['Borough'][n]
    else:
        df_clean['Neighborhood'][n] = df_clean['Neighborhood'][n]

# Print the number of rows in the dataframe
print('Total number of rows in the dataframe is', df_clean.shape[0])

Total number of rows in the dataframe is 103


## Part 2: Merging the dataframe based on postal code

#### Change the diretory to access the coordinate file

In [2]:
import os
os. chdir("/Users/kean/Desktop/Capstone_Project")

#### Merge the dataframe by the PostalCode

In [3]:
df_coordinate = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinate['PostalCode'] = df_coordinate['Postal Code']
df_merge = pd.merge(df_clean, df_coordinate, on = ['PostalCode'])
df_merge.drop('Postal Code', axis = 1, inplace = True)
df_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3: Explore and cluster the neighborhoods in Toronto

#### Filter out the Toronto borough

In [4]:
df_Toronto = df_merge[df_merge['Borough'].str.contains('Toronto')]
df_Toronto.reset_index(drop = True, inplace = True)
df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Use geopy library to get the latitude and longitude values of Toronto

In [5]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto city'
geolocator = Nominatim(user_agent="TR_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

In [6]:
import folium
Toronto_map = folium.Map([latitude,longitude], zoom_start = 12)

# add markers to map
for lat, lng, label in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

#### Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'FQ2Y2PCB24OJCCVYUYXWSTS1TQJZCTBJF453K1EPGIJN1PCI' 
CLIENT_SECRET = 'QOV5GXQ3IRAVL5CSOFJU3CZ005DNOKMU0E0ALGD4QB40DO4U'
VERSION = '20180605' # Foursquare API version

#### Create a function to get the top 50 venues within 500m of each enighborhood in Toronto

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius, Limit):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venue = getNearbyVenues(df_Toronto['Neighborhood'], df_Toronto['Latitude'], df_Toronto['Longitude'], 500, 50)

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

#### Check the first 5 rows of the data extracted from Foursquare 

In [66]:
Toronto_venue.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### Analyse each neighborhood

In [67]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venue[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venue['Neighborhood'] 

Toronto_onehot.head()

,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Find the top 5 most common venues for each neighborhood

In [68]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0  American Restaurant  0.08
1           Steakhouse  0.06
2                 Café  0.06
3          Coffee Shop  0.06
4       Breakfast Spot  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.08
1    Cocktail Bar  0.06
2  Farmers Market  0.04
3        Beer Bar  0.04
4      Steakhouse  0.04


----Brockton, Exhibition Place, Parkdale Village----
               venue  freq
0               Café  0.11
1     Breakfast Spot  0.11
2        Coffee Shop  0.11
3  Convenience Store  0.05
4          Pet Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2          Restaurant  0.06
3      Farmers Market  0.06
4             Brewery  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0     Airport Service  0.20
1      Ai

#### Process the data to create a dataframe containing the top 10 venues for each neighborhood

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

# Display the top 10 venues for first 5 neighborhoods
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",American Restaurant,Coffee Shop,Steakhouse,Café,Hotel,Gastropub,Pizza Place,Breakfast Spot,Asian Restaurant,Gym / Fitness Center
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Steakhouse,Beer Bar,Cheese Shop,Café,Bakery,Restaurant,Seafood Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Italian Restaurant,Pet Store,Convenience Store,Climbing Gym,Caribbean Restaurant,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Comic Shop,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Boutique,Sculpture Garden,Plane,Harbor / Marina,Boat or Ferry,Airport Gate,Airport Food Court


#### Run *k*-means to cluster the neighborhood into 3 clusters and include the cluster info into the Toronto master data dataframe

In [70]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)
#neighborhoods_venues_sorted.drop('Cluster Labels', axis = 1, inplace = True)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, init = "k-means++").fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Music Venue,Yoga Studio,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Liquor Store,Café,Juice Bar,Pub
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Ice Cream Shop,Coffee Shop,Pet Store,Movie Theater,Pub,Burrito Place,Burger Joint,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Yoga Studio,Coworking Space,Pizza Place,Park,Music Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Construction & Landscaping,Swim School,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run


#### Plot the Toronto map with the cluster data 

In [71]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyse the cluster data

In [120]:
#### Cluster 1 Neighborhood

In [125]:
Toronto_cluster_1 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
print(Toronto_cluster_1['1st Most Common Venue'].value_counts().index.tolist()[0] , 
      'is ranked the most in 1st most common venue in the cluster 1 neighborhood with total' , 
      Toronto_cluster_1['1st Most Common Venue'].value_counts()[0] , 'counts')
print()
print(Toronto_cluster_1['2nd Most Common Venue'].value_counts().index.tolist()[0] , 
      'is ranked the most in 2nd most common venue in the cluster 1 neighborhood with total' , 
      Toronto_cluster_1['2nd Most Common Venue'].value_counts()[0] , 'counts')
print()

Toronto_cluster_1

Coffee Shop is ranked the most in 1st most common venue in the cluster 1 neighborhood with total 12 counts

Café is ranked the most in 2nd most common venue in the cluster 1 neighborhood with total 8 counts



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Pub,Music Venue,Yoga Studio,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Liquor Store,Café,Juice Bar,Pub
2,East Toronto,0,Park,Sandwich Place,Ice Cream Shop,Coffee Shop,Pet Store,Movie Theater,Pub,Burrito Place,Burger Joint,Brewery
3,East Toronto,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Yoga Studio,Coworking Space,Pizza Place,Park,Music Store
4,Central Toronto,0,Bus Line,Park,Construction & Landscaping,Swim School,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run
5,Central Toronto,0,Gym,Clothing Store,Park,Breakfast Spot,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Electronics Store,Eastern European Restaurant
6,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant,Furniture / Home Store,Metro Station
7,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Pharmacy,Dance Studio
9,Central Toronto,0,Coffee Shop,Pub,Supermarket,Fried Chicken Joint,Vietnamese Restaurant,Pizza Place,Light Rail Station,Bagel Shop,American Restaurant,Sushi Restaurant
11,Downtown Toronto,0,Coffee Shop,Bakery,Pizza Place,Café,Pub,Market,Restaurant,Italian Restaurant,Pet Store,Japanese Restaurant


#### Cluster 2 Neighborhood

In [126]:
Toronto_cluster_2 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
print(Toronto_cluster_2['1st Most Common Venue'].value_counts().index.tolist()[0] , 
      'is ranked the most in 1st most common venue in the cluster 2 neighborhood with total' , 
      Toronto_cluster_2['1st Most Common Venue'].value_counts()[0] , 'counts')
print()
print(Toronto_cluster_2['2nd Most Common Venue'].value_counts().index.tolist()[0] , 
      'is ranked the most in 2nd most common venue in the cluster 2 neighborhood with total' , 
      Toronto_cluster_2['2nd Most Common Venue'].value_counts()[0] , 'counts')
print()
Toronto_cluster_2

Garden is ranked the most in 1st most common venue in the cluster 2 neighborhood with total 1 counts

Yoga Studio is ranked the most in 2nd most common venue in the cluster 2 neighborhood with total 1 counts



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Garden,Yoga Studio,Dance Studio,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store


#### Cluster 3 Neighborhood

In [127]:
Toronto_cluster_3 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
print(Toronto_cluster_3['1st Most Common Venue'].value_counts().index.tolist()[0] , 
      'is ranked the most in 1st most common venue in the cluster 3 neighborhood with total' , 
      Toronto_cluster_3['1st Most Common Venue'].value_counts()[0] , 'counts')
print()
print(Toronto_cluster_3['2nd Most Common Venue'].value_counts().index.tolist()[0] , 
      'is ranked the most in 2nd most common venue in the cluster 3 neighborhood with total' , 
      Toronto_cluster_3['2nd Most Common Venue'].value_counts()[0] , 'counts')
print()
Toronto_cluster_3

Playground is ranked the most in 1st most common venue in the cluster 3 neighborhood with total 1 counts

Trail is ranked the most in 2nd most common venue in the cluster 3 neighborhood with total 1 counts



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,2,Playground,Trail,Tennis Court,Cuban Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
10,Downtown Toronto,2,Park,Playground,Trail,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
